In [1]:
import os, glob
import numpy as np
import pandas as pd
from scipy.stats import hypergeom

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import scipy.stats

In [2]:


base_dir_name = '/Users/ndeveaux/Dropbox (Simons Foundation)/HIV_Collaboration_2016/2017_activities/outlier_kept_in_full_model_100_subsamples'
dir_name = os.path.join(base_dir_name, 'pval1_quartile_priors')
betas_75k = pd.read_csv(os.path.join(dir_name, 'network_1_to_399_by_betas_thresholded_at_75000_edges.tsv'), sep = '\t')

In [3]:
betas_75k.head()

,ens_tf,ens_target,beta.sign.sum,prior,beta.sign.sum.abs,regulator,target
0,ENSG00000168310,ENSG00000128383,202.751469,382,202.751469,IRF2,APOBEC3A
1,ENSG00000168310,ENSG00000162654,194.005437,376,194.005437,IRF2,GBP4
2,ENSG00000170581,ENSG00000149131,185.525720,388,185.525720,STAT2,SERPING1
3,ENSG00000168310,ENSG00000010030,183.575827,389,183.575827,IRF2,ETV7
4,ENSG00000185507,ENSG00000162654,179.911598,385,179.911598,IRF7,GBP4


In [4]:
sparse_df = pd.read_csv(os.path.join(dir_name, 'all_ones_prior_from_network_1_to_399_by_betas_thresholded_at_75000_edges.tsv'), sep = '\t', index_col = 0)

In [5]:
class Network(object):
    def __init__(self, network_df):
        self.topology = None
        self.symbol_lookup = None
        self.id_lookup = None
        self.target_set = None
        self.tf_set = None
        self.load(network_df)
        
    def load(self, df):
        self.target_set = set(df.ens_target)
        self.tf_set = set(df.ens_tf)
        self.topology = {}
        self.symbol_lookup = {}
        self.id_lookup = {}
        source_key = None
        
        try:
            test = row['source']
            source_key = 'source'
        except:
            source_key = 'regulator'
            
        for idx, row in df.iterrows():
            tf_id = row['ens_tf']
            tf_sym = row[source_key]
            target_id = row['ens_target']
            target_sym = row['target']

            try:
                self.topology[tf_id].add(target_id)
            except:
                self.topology[tf_id] = set([target_id])

            self.symbol_lookup[tf_id] = tf_sym
            self.symbol_lookup[target_id] = target_sym
            self.id_lookup[tf_sym] = tf_id
            self.id_lookup[target_sym] = target_id

In [6]:
def make_graph(network_path):
    
    network_df = pd.read_table(network_path)
    network = Network(network_df)
    
    node_set = network.target_set | network.tf_set
    node_order = list(node_set)
    sym_order = [network.symbol_lookup[x] for x in node_order]
    num_nodes = len(node_order)
    
    U = nx.Graph()
    G = nx.DiGraph()
    for idx, ens in enumerate(node_order):
        attributes = {}
        attributes['label'] = sym_order[idx]
        G.add_node(ens, attributes)
        U.add_node(ens, attributes)
        
    for tf in network.topology:
        for target in network.topology[tf]:
            G.add_edge(tf, target, {
                'weight': 1.0,
                       })
            U.add_edge(tf, target, {})
            
    return G, U

In [7]:
network_path = os.path.join(dir_name, 'network_1_to_399_by_betas_thresholded_at_75000_edges.tsv')
directed_G, undirected_g = make_graph(network_path)
shortest_paths = nx.all_pairs_dijkstra_path_length(undirected_g)

k = len(shortest_paths)
apsp = np.zeros([k,k])

order = list(paths.keys())
indexer = {}
for idx, ens in enumerate(order):
    indexer[ens] = idx
    
for ens_i, row_i in paths.items():
    for ens_j, distance in row_i.items():
        i = indexer[ens_i]
        j = indexer[ens_j]
        apsp[i,j] = distance
        
undirected_shortest_paths = pd.DataFrame(apsp, index=order, columns=order)

NameError: name 'paths' is not defined

In [1]:
directed_G

NameError: name 'directed_G' is not defined